In [8]:
import openai
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import sys
import re
import time
import glob

import os
import openai

### Convert species data into something readable for the machine

In [21]:
# function for conversion
def conversionFunction(file_path):
        
    # read the contents
    with open(file_path, 'r') as file:
        file_content = file.read()

    # replace newlines with '.'
    modified_file_content = file_content.replace('\n',' ')

    # Extracting the file name from the path (without the extension)
    file_name = file_path.split('/')[-1].split('.')[0]

    # reformat the text
    reformatted_modified_text = f"{file_name}|{modified_file_content}"

    # path for new reformatted file
    new_file_path = "Lineas/reformatted_txt/reformatted_"+file_name+".txt"

    # writing the reformatted text to a new file
    with open(new_file_path,'w') as new_file:
        new_file.write(reformatted_modified_text)


import glob
folder_path = "Lineas\\txt\\"
txt_files = glob.glob(f"{folder_path}/*.txt")
txt_files = [file.replace('\\', '/') for file in txt_files]
# run all files through the conversionFunction
for txt_file in txt_files:
    conversionFunction(txt_file)


In [10]:
# we need to divide the text into 4 equal parts
def divide_text_into_parts(text, num_parts=10):
    part_length = len(text) // num_parts
    parts = []
    last_index = 0

    for _ in range(num_parts - 1):
        split_index = last_index + part_length
        # Move the split_index to the end of a sentence if needed
        while split_index < len(text) and text[split_index] not in ".?!":
            split_index += 1
        parts.append(text[last_index:split_index + 1])
        last_index = split_index + 1

    # Add the last part
    parts.append(text[last_index:])
    
    return parts

### we check if any files exceeds the amount of text

In [ ]:
#import glob
#folder_path = "Lineas\\reformatted_txt\\"
#txt_files = glob.glob(f"{folder_path}/*.txt")
#txt_files = [file.replace('\\', '/') for file in txt_files]
# run all files through the conversionFunction
#print(txt_files)
#go = True
#if go:
#        for txt_file in txt_files:
#                if txt_file == 'Lineas/reformatted_txt/reformatted_pmcA1036069.txt':
#                        # we get a sample text out
#                        with open(txt_file, 'r') as file:
#                                file_content = file.read()
#                        only_text = file_content.split('|')[1]
#                        only_name_of_file = file_content.split('|')[0]
#
#                        # we need to divide the text into 4 quarters to ask CHATGPT
#                        parts = divide_text_into_parts(only_text)

#                        for part in parts:
#                                print(part)
# 



### we attempt to divide the text into 10 parts and send to chatgpt

In [11]:
def write_or_append_to_file(file_path, text, append=False):
    """
    Write or append text to a file.

    :param file_path: The path to the file.
    :param text: The text to be written or appended.
    :param append: If True, the function appends text to the file. If False, the function overwrites the file.
    """
    mode = 'a' if append else 'w'
    with open(file_path, mode) as file:
        file.write(text + "\n")

In [12]:
import glob
folder_path = "Lineas\\reformatted_txt\\"
result_folder_path = "results_chatgpt-4/species/"
txt_files = glob.glob(f"{folder_path}/*.txt")
txt_files = [file.replace('\\', '/') for file in txt_files]
# run all files through the conversionFunction
# Create a new file where we save the results
go = False
sorted_txt_files = sorted(txt_files)
print(sorted_txt_files)
for txt_file in sorted_txt_files:
        # we get a sample text out
        with open(txt_file, 'r') as file:
                file_content = file.read()
        only_text = file_content.split('|')[1]
        only_name_of_file = file_content.split('|')[0]

        # we need to divide the text into 10 quarters to ask CHATGPT
        parts = divide_text_into_parts(only_text)
        count_limit = 0
        is_it_first = True
        if only_name_of_file == 'pmcA2599109':
                go = True
        print(only_name_of_file)
        for part in parts:
                if go:
                        try:
                                chat = openai.ChatCompletion.create(
                                model="gpt-4",
                                messages=[
                                        {"role": "system", "content": "You are a NER tool"},
                                        {"role": "assistant", "content": "Extract only species entities and find their correct NCBI ids,display the result only in this format name_of_species|NCBI_id without headers and additional text.Text is: \""+part+"\""},
                                ]
                                )
                                reply = chat.choices[0].message
                                if is_it_first:
                                        write_or_append_to_file(result_folder_path+"results_"+only_name_of_file+".txt",reply.content)
                                        is_it_first = False
                                else:
                                        write_or_append_to_file(result_folder_path+"results_"+only_name_of_file+".txt",reply.content,True)
                                count_limit = count_limit + 1
                        except Exception as e:
                                print("EXCEPTION HAPPENED")
                                print(e)
                                time.sleep(60)
                                count_limit = 0
                                chat = openai.ChatCompletion.create(
                                model="gpt-4",
                                messages=[
                                        {"role": "system", "content": "You are a NER tool"},
                                        {"role": "assistant", "content": "Extract only species entities and find their correct NCBI ids,display the result only in this format name_of_species|NCBI_id without headers and additional text.Text is: \""+part+"\""},
                                ]
                                )
                                reply = chat.choices[0].message
                                if is_it_first:
                                        write_or_append_to_file(result_folder_path+"results_"+only_name_of_file+".txt",reply.content)
                                        is_it_first = False
                                else:
                                        write_or_append_to_file(result_folder_path+"results_"+only_name_of_file+".txt",reply.content,True)
                                count_limit = count_limit + 1
                if count_limit == 450:
                        time.sleep(60)
                        count_limit = 0

['Lineas/reformatted_txt/reformatted_pmcA102792.txt', 'Lineas/reformatted_txt/reformatted_pmcA1036069.txt', 'Lineas/reformatted_txt/reformatted_pmcA1075922.txt', 'Lineas/reformatted_txt/reformatted_pmcA1079799.txt', 'Lineas/reformatted_txt/reformatted_pmcA1131934.txt', 'Lineas/reformatted_txt/reformatted_pmcA1190194.txt', 'Lineas/reformatted_txt/reformatted_pmcA1239921.txt', 'Lineas/reformatted_txt/reformatted_pmcA1257442.txt', 'Lineas/reformatted_txt/reformatted_pmcA1312363.txt', 'Lineas/reformatted_txt/reformatted_pmcA1347483.txt', 'Lineas/reformatted_txt/reformatted_pmcA1352346.txt', 'Lineas/reformatted_txt/reformatted_pmcA140144.txt', 'Lineas/reformatted_txt/reformatted_pmcA140510.txt', 'Lineas/reformatted_txt/reformatted_pmcA1459157.txt', 'Lineas/reformatted_txt/reformatted_pmcA147033.txt', 'Lineas/reformatted_txt/reformatted_pmcA1474674.txt', 'Lineas/reformatted_txt/reformatted_pmcA150219.txt', 'Lineas/reformatted_txt/reformatted_pmcA1521150.txt', 'Lineas/reformatted_txt/reformat

# Data preprocessing

In [10]:
# first we restructure the entire results into one file the same way the Lineas file is structured
result_folder_path = "results_chatgpt-4/species/original/"
# get all file names
result_file_names = []
new_file_path = "results_chatgpt-4/species/compressed/Lineas_true_results.csv"
# Loop through the directory
for filename in os.listdir(result_folder_path):
    if os.path.isfile(os.path.join(result_folder_path, filename)):
        result_file_names.append(os.path.join(result_folder_path, filename))


dataFrames = []     

def clean_ncbi_id(ncbi_id):
    # Convert NCBI_id to string in case it's not
    ncbi_id_str = str(ncbi_id)
    # Remove '.0' only if the value is numeric and ends with '.0'
    if ncbi_id_str.replace('.', '', 1).isdigit() and ncbi_id_str.endswith('.0'):
        return ncbi_id_str[:-2]
    return ncbi_id_str


# Loop through each file in the directory
for filename in result_file_names:
    file_path = filename
    print(filename)
    first_split = filename.split('/')[3]
    actual_file_name = first_split.split('_')[1]
    just_the_name = actual_file_name.split('.')[0]
    # Check if it's a file
    if os.path.isfile(file_path):
        # Read the file into a DataFrame, ensuring NCBI_id is read as a string
        df = pd.read_csv(file_path, sep='|', header=None, names=['name_of_species', 'NCBI_id'], dtype={'NCBI_id': str})

        # Modify the NCBI_id column
        df['NCBI_id'] = df['NCBI_id'].apply(lambda x: 'species:ncbi:' + str(x))

        # Add a new column with the filename
        df['name_of_file'] = just_the_name

        # Append the DataFrame to the list
        dataFrames.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataFrames)

# Reorder columns
combined_df = combined_df[['name_of_file', 'name_of_species', 'NCBI_id']]

# Write the combined DataFrame to a new CSV file
combined_df.to_csv(new_file_path, sep='|', index=False)

results_chatgpt-4/species/original/results_pmcA102792.txt
results_chatgpt-4/species/original/results_pmcA1036069.txt
results_chatgpt-4/species/original/results_pmcA1075922.txt
results_chatgpt-4/species/original/results_pmcA1079799.txt
results_chatgpt-4/species/original/results_pmcA1131934.txt
results_chatgpt-4/species/original/results_pmcA1190194.txt
results_chatgpt-4/species/original/results_pmcA1239921.txt
results_chatgpt-4/species/original/results_pmcA1257442.txt
results_chatgpt-4/species/original/results_pmcA1312363.txt
results_chatgpt-4/species/original/results_pmcA1347483.txt
results_chatgpt-4/species/original/results_pmcA1352346.txt
results_chatgpt-4/species/original/results_pmcA140144.txt
results_chatgpt-4/species/original/results_pmcA140510.txt
results_chatgpt-4/species/original/results_pmcA1459157.txt
results_chatgpt-4/species/original/results_pmcA147033.txt
results_chatgpt-4/species/original/results_pmcA1474674.txt
results_chatgpt-4/species/original/results_pmcA150219.txt
re

### CHATGPT-3.5 preprocessing

In [40]:
# --- cleaned results chatGPT ---
article_results = pd.read_csv('./results_chatGPT-3_5/species/results_reformated/Lineas_true_results.csv',sep='|')

dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_ncbi = {}
current_file = None
chatGPT_annotations = set()
chatGPT_annotations_2 = set()
for index,row in article_results.iterrows():
    file_name = row[0]
    if file_name != current_file:
        current_file = file_name
        chatGPT_annotations = set()
        chatGPT_annotations_2 = set()
    string_row = str(row[2])
    chatGPT_annotations.add((str(row[1]).strip(),string_row.strip()))
    chatGPT_annotations_2.add(str(row[1]).strip())
    dictionary_of_chatGPT_results[current_file] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_ncbi[current_file] = chatGPT_annotations_2

# --- End get CHATGPT results ---
    
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0

# --- start get CORPORA results ---

lineus_corpora = pd.read_csv('./Lineas/tags.tsv',sep='\t')
dictionary_of_results = {}
dictionary_of_results_without_ncbi = {}

#current_file = None
annotations = set()
annotations_2 = set()
current_file = None
all_correct_entities = 0

for index,row in lineus_corpora.iterrows():
    file_name = row[1]
    if file_name != current_file:
        current_file = file_name
        annotations = set()
        annotations_2 = set()
    annotations.add((row[4].strip(),row[0].strip()))
    annotations_2.add(row[4])
    dictionary_of_results[current_file] = annotations
    dictionary_of_results_without_ncbi[current_file] = annotations_2
    all_correct_entities += len(annotations_2)
    


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
what_is_missing2 = []
for element in dictionary_of_results_without_ncbi:
    does_it_exist = False
    for element2 in dictionary_of_chatGPT_results_without_ncbi:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

for element in dictionary_of_chatGPT_results_without_ncbi:
    does_it_exist = False
    for element2 in dictionary_of_results_without_ncbi:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing2.append(element)
         
for element in what_is_missing:
    dictionary_of_chatGPT_results[element] = set()
    dictionary_of_chatGPT_results_without_ncbi[element] = set()
for element in what_is_missing2:
    dictionary_of_results[element] = set()
    dictionary_of_results_without_ncbi[element] = set()

print(len(dictionary_of_results_without_ncbi))
print(len(dictionary_of_chatGPT_results_without_ncbi))
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results))

# --- end get CORPORA results ---
    
path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/species/chatGPT_species_statistics_without_NCBI.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/species/chatGPT_species_statistics_with_NCBI.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/species/confusion_matrix_species_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/species/confusion_matrix_species_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/species/confusion_matrix_species_entities_test_contains_and_chatgpt_contains.csv"


100
100
100
100


### Chatgpt-4 preprocessing

In [55]:
# --- cleaned results chatGPT ---
article_results = pd.read_csv('./results_chatGPT-4/species/compressed/Lineas_true_results.csv',sep='|')

dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_ncbi = {}
current_file = None
chatGPT_annotations = set()
chatGPT_annotations_2 = set()
for index,row in article_results.iterrows():
    file_name = row[0]
    if file_name != current_file:
        current_file = file_name
        chatGPT_annotations = set()
        chatGPT_annotations_2 = set()
    string_row = str(row[2])
    chatGPT_annotations.add((str(row[1]).strip(),string_row.strip()))
    chatGPT_annotations_2.add(str(row[1]).strip())
    dictionary_of_chatGPT_results[current_file] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_ncbi[current_file] = chatGPT_annotations_2

# --- End get CHATGPT results ---
    
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0

# --- start get CORPORA results ---

lineus_corpora = pd.read_csv('./Lineas/tags.tsv',sep='\t')
dictionary_of_results = {}
dictionary_of_results_without_ncbi = {}

#current_file = None
annotations = set()
annotations_2 = set()
current_file = None
all_correct_entities = 0

for index,row in lineus_corpora.iterrows():
    file_name = row[1]
    if file_name != current_file:
        current_file = file_name
        annotations = set()
        annotations_2 = set()
    annotations.add((row[4].strip(),row[0].strip()))
    annotations_2.add(row[4])
    dictionary_of_results[current_file] = annotations
    dictionary_of_results_without_ncbi[current_file] = annotations_2
    all_correct_entities += len(annotations_2)

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))    

# Some of them are missing so we need to add them with empty array:
what_is_missing = []
what_is_missing2 = []
for element in dictionary_of_results_without_ncbi:
    does_it_exist = False
    for element2 in dictionary_of_chatGPT_results_without_ncbi:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

for element in dictionary_of_chatGPT_results_without_ncbi:
    does_it_exist = False
    for element2 in dictionary_of_results_without_ncbi:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing2.append(element)
         
for element in what_is_missing:
    dictionary_of_chatGPT_results[element] = set()
    dictionary_of_chatGPT_results_without_ncbi[element] = set()
for element in what_is_missing2:
    dictionary_of_results[element] = set()
    dictionary_of_results_without_ncbi[element] = set()

print(len(dictionary_of_results_without_ncbi))
print(len(dictionary_of_chatGPT_results_without_ncbi))
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results))
# --- end get CORPORA results ---
    
path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/species/chatGPT_species_statistics_without_NCBI.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/species/chatGPT_species_statistics_with_NCBI.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/species/confusion_matrix_species_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/species/confusion_matrix_species_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/species/confusion_matrix_species_entities_test_contains_and_chatgpt_contains.csv"


72
95
96
96
96
96


# Analysis of the data (Only entities)
_____

### PURE CORRECT ENTITIES

In [56]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results_without_ncbi[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_ncbi[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper():
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 397
All correct entities: 36393
Percentage(%): 1.090
Number of excess entities (incorrect): 382


### If the test entity is contained in the CHATGPT entity

In [57]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results_without_ncbi[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_ncbi[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper():
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 433
All correct entities: 36393
Percentage(%): 1.189
Number of excess entities (incorrect): 346


### If test entities can be found in CHATGPTS response and vice versa

In [58]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results_without_ncbi[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_ncbi[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper() or actual.strip().upper() in test.upper():
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 460
All correct entities: 36393
Percentage(%): 1.263
Number of excess entities (incorrect): 319


# Analysis of data (with NCBI)
____

## Pure correct entities

In [59]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual[0].strip().upper() == test[0].upper() and actual[1] == test[1]:
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 260
All correct entities: 36393
Percentage(%): 0.714
Number of excess entities (incorrect): 665


### If the test entity is contained in the CHATGPT entity

In [60]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper()) and (actual[1] == test[1]):
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 278
All correct entities: 36393
Percentage(%): 0.763
Number of excess entities (incorrect): 647


### If test entities can be found in CHATGPTS response and vice versa

In [61]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_ncbi:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper() or actual[0].strip().upper() in test[0].upper()) and (actual[1] == test[1]):
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 292
All correct entities: 36393
Percentage(%): 0.802
Number of excess entities (incorrect): 633


# Analysis confusion matrix
_____

## Preprocessing

In [62]:
### Create correct dictionaries
pairs_test_set = set()
pairs_chatGPT_set = []
unique_chatGPT_ids = set()
unique_test_set = set()

# get all meshes from CHATGPT and their entities
for key in dictionary_of_chatGPT_results:
    for ncbi_id in dictionary_of_chatGPT_results[key]:
        pairs_chatGPT_set.append((ncbi_id[0],ncbi_id[1]))
        unique_chatGPT_ids.add(ncbi_id[1])

# get all unique meshes from test set
for key in dictionary_of_results:
    for ncbi_id in dictionary_of_results[key]:
        pairs_test_set.add((ncbi_id[0],ncbi_id[1]))
        unique_test_set.add(ncbi_id[1])

list_of_test_meshes = list(pairs_test_set)
unique_test_set = list(unique_test_set)
list_of_chatGPT_MESHES = list(unique_chatGPT_ids)

### Function to create dataframe

In [63]:
def createRowForDataframe(key,array,all_chatGPT_unique_ids):
    array_of_values = {}
    array_of_values["test_NCBI_ID"] = key 
    for element in all_chatGPT_unique_ids:
        counter = 0
        for ground_base_id in array:
            if element == ground_base_id:
                counter += 1
        array_of_values[element] = counter
    return array_of_values


## Pure correct entities

In [64]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0].upper() == entity_set[0].strip().upper():
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_NCBI_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1265887387.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

### Clean confusion matrix and save it into a csv

In [65]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_NCBI_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_pure,index=False)

## If the test entity is contained in the CHATGPT entity

In [66]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0].upper() == entity_set[0].strip().upper() or test_entity[0].upper() in entity_set[0].strip().upper():
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_NCBI_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1917407336.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1917407336.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\1917407336.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

### Clean confusion matrix and save it into a csv

In [67]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_NCBI_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_lapover,index=False)

### If test entities can be found in CHATGPTS response and vice versa

In [68]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0].upper() == entity_set[0].strip().upper() or test_entity[0].upper() in entity_set[0].strip().upper() or entity_set[0].strip().upper() in test_entity[0].upper():
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_NCBI_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_6432\3118245838.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\3118245838.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_6432\3118245838.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

### Clean confusion matrix and save it into a csv

In [69]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_NCBI_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_full_lapover,index=False)